In [4]:
pip install unidecode beautifulsoup4 python-magic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-magic]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
coded_string = '''Q5YACgA...'''
# Add padding to the Base64 string if necessary
missing_padding = len(coded_string) % 4
if missing_padding:
	coded_string += '=' * (4 - missing_padding)

# Decode the Base64 string
decoded_data = base64.b64decode(coded_string)
decoded_data


b'C\x96\x00\n\x00'

In [3]:
def decode_hexstring(hexstring):
    decoded = ''

    for i in range(0, len(hexstring), 2):
        b = hexstring[i:i+2]
        b = b.decode() # it's a byte-string

        try:
            c = bytes.fromhex(b).decode()
        except: # the last char might be missing
            c = '☐'

        decoded = decoded + c

    return decoded

print(decode_hexstring(b'737030306b792d686578737472696e676'))


sp00ky-hexstring☐


In [ ]:
# ============================
# UTILS
# ============================

def safe_base64_decode(s):
    try:
        return base64.b64decode(s).decode("utf-8")
    except:
        return None

def safe_hex_decode(s):
    try:
        return bytes.fromhex(s).decode("utf-8")
    except:
        return None

def safe_rot13(s):
    try:
        return codecs.decode(s, "rot_13")
    except:
        return None

def decode_rot13_segments(text):
    words = text.split()
    new = []

    for w in words:
        # ROT13 candidates = pure letters
        if re.fullmatch(r"[A-Za-z]{4,}", w):
            decoded = safe_rot13(w)
            # validate: decoded must contain a vowel → looks like real word
            if any(v in decoded.lower() for v in "aeiou"):
                new.append(decoded)
            else:
                new.append(w)
        else:
            new.append(w)

    return " ".join(new)
